In [1]:
from kiteconnect import KiteConnect

In [2]:
import logging
import requests

In [3]:
import pandas as pd

In [4]:
from time import time, sleep

In [5]:
from datetime import datetime, timedelta

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
api_key = "f5ean4aaxq3wv9n3"
client_id = "ED5107"
api_secret = "s1kbs5y654zhbpqgd2rvslqc6shx9yh8"

In [8]:
logging.basicConfig(level=logging.DEBUG)
kite = KiteConnect(api_key=api_key)
print(kite.login_url())

https://kite.trade/connect/login?api_key=f5ean4aaxq3wv9n3&v=3


In [9]:
# Get the Request Token from the redirect URL

request_token = "kxUB29orXpZDVOCpARctWElF0bPoy54t"

# Get Access Token

In [10]:
#data = kite.generate_session(request_token, api_secret=api_secret)
#access_token = data["access_token"]
access_token = "ngIWpCT8mnlRmnBiSV5T32C7EJKZql5E"
kite.set_access_token(access_token)
print(access_token)

ngIWpCT8mnlRmnBiSV5T32C7EJKZql5E


In [11]:
print(kite.profile())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /user/profile HTTP/1.1" 200 None


{'user_id': 'ED5107', 'user_type': 'individual', 'email': 'ramanathannalini@gmail.com', 'user_name': 'Nalini Ramanathan', 'user_shortname': 'Nalini', 'broker': 'ZERODHA', 'exchanges': ['BFO', 'CDS', 'BSE', 'NFO', 'MF', 'NSE'], 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'], 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'], 'avatar_url': '', 'meta': {'demat_consent': 'consent'}}


# NOTE:
# After every request ensure that you sleep for 0.5 second because there is a limit of a maximum of  3 requests per second.

# Placing a trade

In [12]:
def place_order(trading_symbol, exchange, qty, transaction_type, product, order_type, variety='regular',price=None,validity=None,disclosed_quantity=None,trigger_price=None,squareoff=None,stoploss=None,trailing_stoploss=None, tag=None):
    """
    Refer: https://kite.trade/docs/connect/v3/orders/
    
    tradingsymbol Eg RELIANCE, 
    exchange => ['MF', 'NSE', 'NFO', 'CDS', 'BSE', 'BFO'],
    quantity as integer,
    variety is 'regular' by default,
    transaction_type => 'BUY' or 'SELL',
    product Eg 'CNC' for Equity,
    order_type => ['MARKET', 'LIMIT', 'SL', 'SL-M'],
    price=None,
    validity=None,
    disclosed_quantity=None,
    trigger_price=None,
    squareoff=None,
    stoploss=None,
    trailing_stoploss=None,
    tag=None
    """
    order_id = kite.place_order(tradingsymbol=trading_symbol, 
                     exchange=exchange,
                     quantity=qty,
                     variety=variety,
                     transaction_type=transaction_type,
                     product=product,
                     order_type=order_type,
                     price=price,
                     validity=validity,
                     disclosed_quantity=disclosed_quantity,
                     trigger_price=trigger_price,
                     squareoff=squareoff,
                     stoploss=stoploss,
                     trailing_stoploss=trailing_stoploss,
                     tag=tag)
    return order_id

# Get Instrument Tokens

In [13]:
def get_instrument_token(tradingsymbol, instrument_type, exchange):
    instruments = pd.DataFrame(kite.instruments())
    instrument_token = instruments[(instruments['tradingsymbol']==tradingsymbol) & (instruments['instrument_type']==instrument_type) & (instruments['exchange']==exchange)]['instrument_token'].to_list()[0]
    return str(instrument_token)

# Get Historical Data

In [14]:
def get_historical_data(instrument_token, from_date, to_date, interval):
    return pd.DataFrame(kite.historical_data(instrument_token=instrument_token,
                                    from_date=from_date, 
                                   to_date=to_date,
                                   interval=interval))

# Get Simple Moving Average

In [15]:
def get_moving_average(timeseries, window):
    timeseries[str(window)+'d_EMAvg'] = timeseries.iloc[:,0].rolling(window=window).mean()
    return timeseries

# Get Exponential Moving Average

In [16]:
def get_exponential_moving_avg(timeseries, span):
    """
    adjust=False specifies that we are interested in the recursive calculation mode.
    """
    timeseries[str(span)+'d_EMAvg'] = timeseries.iloc[:,0].ewm(span=span, adjust=False).mean()
    return timeseries

In [17]:
#INSTRUMENT DATA
tradingsymbol = 'BANKNIFTY20SEPFUT'
instrument_type = 'FUT'
exchange = 'NFO'

# TRADE DETAILS
qty = 1
order_type='MARKET'
product='MIS'
variety='regular'

# CHART SPECIFICATIONS
candle_size = 'minute'
k_period = 10
k_smooth_period = 3
d_smooth_period = 3
d_period = 10

# DEPLOYMENT DURATION
deploy_till = datetime(year=2020, month=8, day=28, hour=15 , minute=30 , second=0)

# TEST TRADING PLATFORM

# BANKNIFTY SEP FUT

In [18]:
def place_trade(trade_log, current_time, price, open_price, current_transaction_type):
    if len(trade_log)==0 or trade_log.loc[list(trade_log.index)[-1], 'status']=='closed':
        current_status = 'open'
    elif trade_log.loc[list(trade_log.index)[-1], 'status']=='open':
        current_status = 'closed'
    if price>open_price:
        candle_color = 'green'
    else:
        candle_color = 'red'
    print(current_time, current_transaction_type, price, current_status)
    trade_log.loc[len(trade_log)] = [current_time, open_price, price, candle_color, current_transaction_type, current_status]
    return trade_log, current_transaction_type, current_status

In [26]:
def create_candle(hist_data, candle_data, prev_ha_open, prev_ha_close, test_end):
    # get OHLC for latest candle
    actual_high = list(hist_data['high'])[-2]
    actual_low = list(hist_data['low'])[-2]
    actual_close = list(hist_data['close'])[-2]
    actual_open = list(hist_data['open'])[-2]
    
    # get Heikin Ashi candle OHLC data
    ha_close = (1/4)*(actual_open+actual_low+actual_high+actual_close)
    if len(candle_data)==0:
        ha_open = (prev_ha_open+prev_ha_close)/2
    else:
        ha_open = (list(candle_data['Open'])[-1] + list(candle_data['Close'])[-1])/2
    ha_high = max([actual_high, ha_open, ha_close])
    ha_low = min([actual_low, ha_open, ha_close])
    if ha_open < ha_close:
        color = 'green'
    else:
        color = 'red'
    
    # Add to candle data
    candle_data.loc[len(candle_data)] = [test_end, ha_open, ha_high, ha_low, ha_close, color]
    
    return candle_data, ha_open, ha_close, actual_open, actual_close

#### 3min 20 threshold

In [20]:
instoken = get_instrument_token(tradingsymbol, instrument_type, exchange)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443
DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments HTTP/1.1" 200 860315


In [24]:
market_open_list = [datetime(2020, 8, 3, 9, 15), datetime(2020, 8, 4, 9, 15), datetime(2020, 8, 5, 9, 15), datetime(2020, 8, 6, 9, 15), datetime(2020, 8, 7, 9, 15), datetime(2020, 8, 10, 9, 15), datetime(2020, 8, 11, 9, 15), datetime(2020, 8, 12, 9, 15), datetime(2020, 8, 13, 9, 15), datetime(2020, 8, 14, 9, 15), datetime(2020, 8, 17, 9, 15), datetime(2020, 8, 18, 9, 15), datetime(2020, 8, 19, 9, 15), datetime(2020, 8, 20, 9, 15), datetime(2020, 8, 21, 9, 15), datetime(2020, 8, 24, 9, 15), datetime(2020, 8, 25, 9, 15), datetime(2020, 8, 26, 9, 15), datetime(2020, 8, 27, 9, 15), datetime(2020, 8, 28, 9, 15), datetime(2020, 8, 31, 9, 15), datetime(2020, 9, 1, 9, 15), datetime(2020, 9, 2, 9, 15), datetime(2020, 9, 3, 9, 15), datetime(2020, 9, 4, 9, 15), datetime(2020, 9, 7, 9, 15)]
prev_ha_open_list = [21686.94, 21177.39, 21551.80, 21594.74, 21649.51, 21775.23, 21967.48, 22311.66, 22305.97, 22207.62, 21700.99, 21786.39, 22200.53, 22331.14, 21959.05, 22295.33, 22797.81, 23068.40, 23406.32, 23588.10, 24586.29, 23775.86, 23953.24, 23926.91, 23579.45, 23062.09]
prev_ha_close_list = [21694.29, 21189.78, 21551.92, 21603.99, 21667.74, 21784.06, 21974.86, 22318.16, 22287.79, 22212.36, 21689.09, 21795.99, 22202.27, 22332.83, 21942.81, 22306.20, 22799.25, 23073.38, 23435.25, 23601.64, 24620.04, 23662.64, 23960.17, 23920.51, 23579.95, 23076.20]

thickness_threshold = 19
interval = '3minute'
candle = 3
last_candle = '15:15:00'

In [38]:
market_open_list = [datetime(2020, 8, 24, 9, 15), datetime(2020, 8, 25, 9, 15), datetime(2020, 8, 26, 9, 15), datetime(2020, 8, 27, 9, 15), datetime(2020, 8, 28, 9, 15), datetime(2020, 8, 31, 9, 15), datetime(2020, 9, 1, 9, 15), datetime(2020, 9, 2, 9, 15), datetime(2020, 9, 3, 9, 15), datetime(2020, 9, 4, 9, 15)]
prev_ha_open_list = [22289.66, 22812.21, 23062.48, 23391.92, 23596.02, 24539.45, 23785.86, 23938.91, 23922.92, 23544.22]
prev_ha_close_list = [22304.97, 22799.22, 23073.80, 23425.33, 23599.97, 24623.20, 23705.37, 23954.75, 23926.0, 23572.85]

thickness_threshold = 25
interval = '5minute'
candle = 5
last_candle = '15:25:00'

In [32]:
market_open_list = [datetime(2020, 8, 26, 9, 15), datetime(2020, 8, 27, 9, 15), datetime(2020, 9, 1, 9, 15), datetime(2020, 9, 3, 9, 15), datetime(2020, 9, 7, 9, 15)]
prev_ha_open_list = [23068.40, 23406.32, 23775.86, 23926.91, 23062.09]
prev_ha_close_list = [23073.38, 23435.25, 23662.64, 23920.51, 23076.20]

thickness_threshold = 19
interval = '3minute'
candle = 3
last_candle = '15:27:00'

In [25]:
for i in range(0,len(market_open_list)):
    market_open = market_open_list[i]
    prev_ha_open = prev_ha_open_list[i]
    prev_ha_close = prev_ha_close_list[i]
    trade_log = pd.DataFrame(columns=['time','open_price','price','candle_color','transaction_type','status'])
    candle_data = pd.DataFrame(columns=['Candle','Open','High','Low','Close','Color'])
    current_transaction_type = None
    current_status = 'closed'
    for delta in range(candle, 380, candle):
        sleep(1)
        test_end = market_open + timedelta(minutes=delta)
        print(test_end)
        test_start = test_end - timedelta(days=1, hours=1)
        for _ in range(5):
            try:
                hist_data = get_historical_data(instoken, test_start, test_end, interval)
                break
            except requests.exceptions.ReadTimeout as e:
                logging.exception("Timeout Error - while getting ltp: \n %s", e)
                logging.info("Retrying")
                continue
        
        #get HA candle data from hist_data
        candle_data, ha_open, ha_close, actual_open, actual_close = create_candle(hist_data, candle_data, prev_ha_open, prev_ha_close, test_end)
        
        # if EOD then exit
        if datetime.strftime(test_end, "%H:%M:%S")==last_candle:
            if current_status=='open':
                if current_transaction_type=='buy':
                    transaction_type='sell'
                    trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
                    break
                if current_transaction_type=='sell':
                    transaction_type='buy'
                    trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
                    break
            else:
                break
        
        #if red candle with thickness then sell if closed/open buy, do nothing if already sell
        if ha_open>ha_close and abs(ha_open-ha_close)>thickness_threshold:
            transaction_type='sell'
            if current_status=='closed':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
            elif current_status=='open' and current_transaction_type=='buy':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
        
        #if green candle with thickness then buy if closed/open sell, do nothing if already buy
        if ha_open<ha_close and abs(ha_open-ha_close)>thickness_threshold:
            transaction_type='buy'
            if current_status=='closed':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
            elif current_status=='open' and current_transaction_type=='sell':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
    
    
    trade_log.to_excel('HA_Candle_Reading-Test_Outputs\_'+interval+'\Trade_Logs\Trade_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'19.xlsx')
    candle_data.to_excel('HA_Candle_Reading-Test_Outputs\_'+interval+'\Candle_Logs\Candle_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'19.xlsx')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-08-03 09:15:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-08-02+08%3A15%3A00&to=2020-08-03+09%3A15%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 None


2020-08-03 09:15:00 sell 21427.65 open


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


...

2020-09-07 15:12:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-06+14%3A12%3A00&to=2020-09-07+15%3A12%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2490
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


2020-09-07 15:15:00


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/14956290/3minute?instrument_token=14956290&from=2020-09-06+14%3A15%3A00&to=2020-09-07+15%3A15%3A00&interval=3minute&continuous=0&oi=0 HTTP/1.1" 200 2523


2020-09-07 15:15:00 sell 23016.55 closed


In [ ]:
market_open_list = [datetime(2020, 8, 3, 10), datetime(2020, 8, 4, 10), datetime(2020, 8, 5, 10), datetime(2020, 8, 6, 10), datetime(2020, 8, 7, 10), datetime(2020, 8, 10, 10), datetime(2020, 8, 11, 10), datetime(2020, 8, 12, 10), datetime(2020, 8, 13, 10), datetime(2020, 8, 14, 10), datetime(2020, 8, 17, 10), datetime(2020, 8, 18, 10), datetime(2020, 8, 19, 10), datetime(2020, 8, 20, 10), datetime(2020, 8, 21, 10), datetime(2020, 8, 24, 10), datetime(2020, 8, 25, 10), datetime(2020, 8, 26, 10), datetime(2020, 8, 27, 10), datetime(2020, 8, 28, 10), datetime(2020, 8, 31, 10), datetime(2020, 9, 1, 10), datetime(2020, 9, 2, 10), datetime(2020, 9, 3, 10), datetime(2020, 9, 4, 10), datetime(2020, 9, 7, 10)]
prev_ha_open_list = [21686.94, 21177.39, 21551.80, 21594.74, 21649.51, 21775.23, 21967.48, 22311.66, 22305.97, 22207.62, 21700.99, 21786.39, 22200.53, 22331.14, 21959.05, 22295.33, 22797.81, 23068.40, 23406.32, 23588.10, 24586.29, 23775.86, 23953.24, 23926.91, 23579.45, 23062.09]
prev_ha_close_list = [21694.29, 21189.78, 21551.92, 21603.99, 21667.74, 21784.06, 21974.86, 22318.16, 22287.79, 22212.36, 21689.09, 21795.99, 22202.27, 22332.83, 21942.81, 22306.20, 22799.25, 23073.38, 23435.25, 23601.64, 24620.04, 23662.64, 23960.17, 23920.51, 23579.95, 23076.20]

entry_thickness_threshold = 20
interval = '3minute'
candle = 3
last_candle = '15:15:00'

SCENARIOS:
    if current_status = closed:
        if green and entry_thickness_threshold>20:
            buy
        if red and entry_thickness_threshold>20:
            sell
    if current_status = open:
        if exit_thickness_threshold<15:
            exit current trade

In [ ]:
for i in range(0,len(market_open_list)):
    market_open = market_open_list[i]
    prev_ha_open = prev_ha_open_list[i]
    prev_ha_close = prev_ha_close_list[i]
    trade_log = pd.DataFrame(columns=['time','open_price','price','candle_color','transaction_type','status'])
    candle_data = pd.DataFrame(columns=['Candle','Open','High','Low','Close','Color'])
    current_transaction_type = None
    current_status = 'closed'
    for delta in range(0, 380, candle):
        sleep(1)
        test_end = market_open + timedelta(minutes=delta)
        print(test_end)
        test_start = test_end - timedelta(days=1, hours=1)
        for _ in range(5):
            try:
                hist_data = get_historical_data(instoken, test_start, test_end, interval)
                break
            except requests.exceptions.ReadTimeout as e:
                logging.exception("Timeout Error - while getting ltp: \n %s", e)
                logging.info("Retrying")
                continue
        
        #get HA candle data from hist_data
        candle_data, ha_open, ha_close, actual_open, actual_close = create_candle(hist_data, candle_data, prev_ha_open, prev_ha_close, test_end)
        
        # if EOD then exit
        if datetime.strftime(test_end, "%H:%M:%S")==last_candle:
            if current_status=='open':
                if current_transaction_type=='buy':
                    transaction_type='sell'
                    trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
                    break
                if current_transaction_type=='sell':
                    transaction_type='buy'
                    trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
                    break
            else:
                break
        
        #if red candle with thickness then sell if closed/open buy, do nothing if already sell
        if current_status=='closed':
            if ha_open>ha_close and abs(ha_open-ha_close)>entry_thickness_threshold:
                transaction_type='sell'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
            if ha_open<ha_close and abs(ha_open-ha_close)>entry_thickness_threshold:
                transaction_type='buy'
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
        if current_status=='open':
            if current_transaction_type=='buy' and ha_open-ha_close:
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
        
        #if green candle with thickness then buy if closed/open sell, do nothing if already buy
        if ha_open<ha_close and abs(ha_open-ha_close)>entry_thickness_threshold:
            transaction_type='buy'
            if current_status=='closed':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
            elif current_status=='open' and current_transaction_type=='sell':
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
                trade_log, current_transaction_type, current_status = place_trade(trade_log, current_time=test_end, price=actual_close, open_price=actual_open, current_transaction_type=transaction_type)
    
    
    trade_log.to_excel('HA_Candle_Reading-Test_Outputs\_'+interval+'\Trade_Logs\Trade_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'20.xlsx')
    candle_data.to_excel('HA_Candle_Reading-Test_Outputs\_'+interval+'\Candle_Logs\Candle_Log_'+datetime.strftime(test_end, "%Y%m%d")+'_'+interval+'20.xlsx')